<a href="https://colab.research.google.com/github/sparks-baird/bayes-opt-particle-packing/blob/main/examples/packing_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install boppf

  Using cached pyro_ppl-1.8.2-py3-none-any.whl (722 kB)
  Using cached kaleido-0.2.1-py2.py3-none-win_amd64.whl (65.9 MB)
     ---------------------------------------- 21.2/21.2 MB 3.3 MB/s eta 0:00:00
  Using cached ax_platform-0.2.6-py3-none-any.whl (1.0 MB)
  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
     ---------------------------------------- 10.0/10.0 MB 3.9 MB/s eta 0:00:00
  Using cached botorch-0.6.6-py3-none-any.whl (387 kB)
  Using cached scikit_learn-1.0.2-cp37-cp37m-win_amd64.whl (7.1 MB)
  Using cached multipledispatch-0.6.0-py3-none-any.whl (11 kB)
  Using cached gpytorch-1.8.1-py2.py3-none-any.whl (361 kB)
     -------------------------------------- 167.3/167.3 MB 3.0 MB/s eta 0:00:00
  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl (14.0 MB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ------------------------------------

In [ ]:
!git clone https://github.com/VasiliBaranov/packing-generation.git

Cloning into 'packing-generation'...
Updating files:  74% (2299/3083)
Updating files:  75% (2313/3083)
Updating files:  76% (2344/3083)
Updating files:  77% (2374/3083)
Updating files:  78% (2405/3083)
Updating files:  79% (2436/3083)
Updating files:  80% (2467/3083)
Updating files:  81% (2498/3083)
Updating files:  82% (2529/3083)
Updating files:  83% (2559/3083)
Updating files:  84% (2590/3083)
Updating files:  85% (2621/3083)
Updating files:  86% (2652/3083)
Updating files:  87% (2683/3083)
Updating files:  88% (2714/3083)
Updating files:  89% (2744/3083)
Updating files:  90% (2775/3083)
Updating files:  91% (2806/3083)
Updating files:  92% (2837/3083)
Updating files:  93% (2868/3083)
Updating files:  94% (2899/3083)
Updating files:  95% (2929/3083)
Updating files:  96% (2960/3083)
Updating files:  97% (2991/3083)
Updating files:  98% (3022/3083)
Updating files:  99% (3053/3083)
Updating files: 100% (3083/3083)
Updating files: 100% (3083/3083), done.


In [ ]:
%cd packing-generation/_Release

C:\Users\sterg\packing-generation\_Release


In [ ]:
%pwd

'C:\\Users\\sterg\\packing-generation\\_Release'

In [ ]:
!make

'make' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
from boppf.utils.data import get_parameters, prep_input_data

In [ ]:
remove_composition_degeneracy = True
remove_scaling_degeneracy = True
(
        subfrac_names,
        params,
        generous_parameters,
        mean_names_out,
        std_names_out,
        orig_mean_names,
        orig_std_names,
) = get_parameters(remove_composition_degeneracy=remove_composition_degeneracy, remove_scaling_degeneracy=remove_scaling_degeneracy)

In [ ]:
params

[{'name': 'mu1_div_mu3',
  'type': 'range',
  'bounds': [0.3333333333333333, 1.6666666666666667],
  'value_type': 'float'},
 {'name': 'mu2_div_mu3',
  'type': 'range',
  'bounds': [0.3333333333333333, 1.6666666666666667],
  'value_type': 'float'},
 {'name': 'std1',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'std2',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'std3',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'comp1',
  'type': 'range',
  'bounds': [0.0, 1.0],
  'value_type': 'float'},
 {'name': 'comp2',
  'type': 'range',
  'bounds': [0.0, 1.0],
  'value_type': 'float'}]

In [ ]:
import plotly.express as px
import numpy as np
s_radii, c_radii, m_fracs = prep_input_data([1.0, 3.0, 5.0], [0.1, 0.1, 0.1], [0.3, 0.3, 0.4], 1e-6, 1000)
px.histogram(s_radii[0] * m_fracs[0])

In [ ]:
from scipy.stats import lognorm
from math import ceil

stds = [0.1, 0.1, 0.1]
means = [1.0, 2.0, 3.0]
comps = [0.3, 0.3, 0.4]

def get_diameters(means, stds, comps, num_particles=100, seed=None):
  samples = []
  for mu, sigma, comp in zip(means, stds, comps):
    samples.append(lognorm.rvs(sigma, scale=mu, size=ceil(comp * num_particles), random_state=seed))

  samples = np.concatenate(samples)[:num_particles]

  if len(samples) != num_particles:
    raise ValueError(f"Number of samples ({len(samples)}) does not match requested number of particles {num_particles}. Ensure `sum(comps)==1` (sum({comps})=={sum(comps)})")

  return samples

samples = get_diameters(means, stds, comps, num_particles=100)
px.histogram(samples)

In [ ]:
from random import randint
import numpy as np
from math import pi
import os
from os import path
import shutil
from uuid import uuid4
from subprocess import DEVNULL, STDOUT, Popen, PIPE, run
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import stat

def write_diameters(X, data_dir=".", uid=str(uuid4())):
  save_dir = path.join(data_dir, uid)
  Path(save_dir).mkdir(exist_ok=True, parents=True)
  fpath = path.join(save_dir, "diameters.txt")
  np.savetxt(fpath, X)

def get_box_length(X, safety_factor = 2.0):
  particles_vol = sum(4/3 * pi * (X/2)**3)
  return (particles_vol * safety_factor)**(1/3)

def run_simulation(flag, util_dir=".", data_dir=".", uid=str(uuid4())):
    exe_name = "PackingGeneration.exe"
    simpath = path.join(util_dir, exe_name)
    new_dir = path.join(data_dir, uid)
    newpath = path.join(new_dir, exe_name)
    shutil.copyfile(simpath, newpath)
    f = Path(newpath)
    f.chmod(f.stat().st_mode | stat.S_IEXEC)
    cwd = os.getcwd()
    os.chdir(new_dir)
    result = run([f"./{exe_name}", flag], capture_output=True, text=True) # stdout=PIPE, stderr=STDOUT
    # print(result.stdout)
    # print(result.stderr)
    os.chdir(cwd)

def particle_packing_simulation(means, stds, comps, num_particles=100, contraction_rate=1e-3, seed=None, data_dir = ".", util_dir = ".", uid = None, safety_factor = 2.0):
  if seed is None:
    seed = randint(0, 100000)
  if uid is None:
    uid = str(uuid4())
    print(uid)

  save_dir = path.join(data_dir, uid)
  Path(save_dir).mkdir(exist_ok=True, parents=True)
  generation_conf_fpath = path.join(save_dir, "generation.conf")
  packing_nfo_fpath = path.join(save_dir, 'packing.nfo')
  packing_xyzd_fpath = path.join(save_dir, 'packing.xyzd')
  
  # X = np.repeat(1.0, num_particles)
  X = get_diameters(means, stds, comps, num_particles=num_particles, seed=seed)

  write_diameters(X, data_dir=data_dir, uid=uid)
  box_length = get_box_length(X, safety_factor = safety_factor)

  # remove existing data files, if any
  names = ["packing.xyzd", "packing_init.xyzd", "packing_prev.xyzd", "contraction_energies.txt", "packing.nfo"]
  [os.remove(path.join(data_dir, uid, name)) if path.exists(name) else None for name in names]

  with open(generation_conf_fpath, "w") as f:
    lines = [
        f"Particles count: {num_particles}",
        f"Packing size: {box_length} {box_length} {box_length}",
        f"Generation start: 1",
        f"Seed: {seed}",
        "Steps to write: 1000",
        "Boundaries mode: 1",
        f"Contraction rate: {contraction_rate}"
        ]
    f.writelines(lines)

  run_simulation("-fba", util_dir=util_dir, data_dir=data_dir, uid=uid)
  
  with open(generation_conf_fpath, "w") as f:
    lines = [
        f"Particles count: {num_particles}",
        f"Packing size: {box_length} {box_length} {box_length}",
        f"Generation start: 0",
        f"Seed: {seed}",
        "Steps to write: 1000",
        "Boundaries mode: 1",
        f"Contraction rate: {contraction_rate}"
        ]
    f.writelines(lines)

  try:
    os.remove(packing_nfo_fpath)
  except Exception as e:
    print(e)
  try:
    run_simulation("-ls", util_dir=util_dir, data_dir=data_dir, uid=uid)
  except Exception as e:
    print(e)

  try:
    os.remove(packing_nfo_fpath)
  except Exception as e:
    print(e)
  try:
    run_simulation("-lsgd", util_dir=util_dir, data_dir=data_dir, uid=uid)
  except Exception as e:
    print(e)

  """https://github.com/VasiliBaranov/packing-generation/issues/30#issue-1103925864"""

  try:
    packing = np.fromfile(path.join(data_dir, uid, 'packing.xyzd')).reshape(-1, 4)
    with open(path.join(data_dir, uid, 'packing.nfo'), "r+") as nfo:
        lines = nfo.readlines()
        Theoretical_Porosity = float(lines[2].split()[2])
        Final_Porosity = float(lines[3].split()[2])
        # print(Theoretical_Porosity, Final_Porosity)

        scaling_factor = ((1 - Final_Porosity) / (1 - Theoretical_Porosity)) ** (1 / 3)

        real_diameters = packing[:, 3] * scaling_factor
        actual_density = sum((4/3) * pi * (np.array(real_diameters) / 2)**3) / box_length**3
        packing[:, 3] = real_diameters
        packing.tofile(packing_xyzd_fpath)        # updating the packing: this line will modifies diameters in the packing.xyzd

        # update packing.nfo and set TheoreticalPorosity to FinalPorosity to avoid scaling the packing once again the next time running this script.
        lines[3] = lines[3].replace(str(Final_Porosity), str(Theoretical_Porosity))
        nfo.seek(0)
        nfo.writelines(lines)

        return actual_density
  except:
    return np.nan

In [ ]:
# stds = [1.0, 1.0, 1.0]
# means = [1.0, 3.0, 5.0]
# comps = [0.3, 0.3, 0.4]
stds = [1.0]
means = [1.0]
comps = [1.0]
samples = get_diameters(means, stds, comps, num_particles=100)
px.histogram(samples)

In [ ]:
print(min(samples), max(samples))

0.053428297740607 11.99241689236166


In [ ]:
%%time
%cd /content/packing-generation/_Release
packing_fractions = [particle_packing_simulation(means, stds, comps, num_particles=100, seed=seed) for seed in range(2)]

/content/packing-generation/_Release
0b474d47-54c7-4225-8db7-e3936f394ea3
1ce1c9aa-c7da-404e-85b5-41d853178880
CPU times: user 181 ms, sys: 84 ms, total: 265 ms
Wall time: 11.1 s


In [ ]:
packing_fractions

[0.685245618260134, 0.6862302448815794]

In [ ]:
params

[{'name': 'mu1_div_mu3',
  'type': 'range',
  'bounds': [0.3333333333333333, 1.6666666666666667],
  'value_type': 'float'},
 {'name': 'mu2_div_mu3',
  'type': 'range',
  'bounds': [0.3333333333333333, 1.6666666666666667],
  'value_type': 'float'},
 {'name': 'std1',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'std2',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'std3',
  'type': 'range',
  'bounds': [0.1, 1.0],
  'value_type': 'float'},
 {'name': 'comp1',
  'type': 'range',
  'bounds': [0.0, 1.0],
  'value_type': 'float'},
 {'name': 'comp2',
  'type': 'range',
  'bounds': [0.0, 1.0],
  'value_type': 'float'}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/packing-generation/_Release
import os
from math import isnan, ceil
from ax.service.ax_client import AxClient
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models
from tqdm.notebook import trange
from ray import tune, remote
import ray
from ray.tune.search.ax import AxSearch

data_dir = path.join("/content/drive", "MyDrive", "sparks-baird", "boppf")

# from boppf.utils.data import mean_names, std_names, frac_names

num_trials = 2 ** 13
batch_size = 2
checkpoint = 200


/content/packing-generation/_Release


In [ ]:
from glob import glob
fpath = path.join(data_dir, f'sobol-[0-9]*-remove_composition_degeneracy={remove_composition_degeneracy},remove_scaling_degeneracy={remove_scaling_degeneracy}-checkpoint.json')
files = glob(fpath)
if len(files) > 0:
  checkpoint_nums = [int(file.split("sobol-")[1].split("-remove_composition_degeneracy")[0]) for file in files]
  latest_checkpoint = files[np.argmax(checkpoint_nums)]
  ax_client = AxClient.load_json_from_file(latest_checkpoint)
else:
  gs = GenerationStrategy(
      steps=[
          # 1. Initialization step (does not require pre-existing data and is well-suited for 
          # initial sampling of the search space)
          GenerationStep(
              model=Models.SOBOL,
              num_trials=num_trials,  # How many trials should be produced from this generation step
              max_parallelism=batch_size,  # Max parallelism for this step
              model_kwargs={"seed": 999},  # Any kwargs you want passed into the model
              model_gen_kwargs={},  # Any kwargs you want passed to `modelbridge.gen`
          ),
      ]
  )

  ax_client = AxClient(generation_strategy=gs)
  ax_client.create_experiment(
      name="boppf_sobol",
      parameters=params,
      objective_name="packing_fraction",
      minimize=False,
      parameter_constraints = ["std1 <= std2", "comp1 + comp2 <= 1.0"],
  )


'/content/drive/MyDrive/sparks-baird/boppf/sobol-801-remove_composition_degeneracy=True,remove_scaling_degeneracy=True-checkpoint.json'

In [ ]:
ray.shutdown()
ray.init(num_cpus=batch_size)

mu3 = 3.0

@remote
def evaluate(parameters):
  # print("current working directory: ", os.getcwd())
  means = [parameters[name] * mu3 for name in ['mu1_div_mu3', 'mu2_div_mu3']]
  means.append(mu3)
  stds = [parameters[name] for name in ["std1", "std2", "std3"]]
  comps = [parameters[name] for name in ["comp1", "comp2"]]
  comps.append(1 - sum(comps))
  try:
    result = particle_packing_simulation(means, stds, comps, num_particles=100)
  except Exception as e:
    print(e)
    result = np.nan
  return {"packing_fraction": result}

# optimization_complete = False
for iter in trange(ceil(num_trials // batch_size)):
    trials_dict, optimization_complete = ax_client.get_next_trials(batch_size)
    print(trials_dict)

    if optimization_complete:
      break

    futures = [evaluate.remote(parameters) for parameters in trials_dict.values()]
    data = ray.get(futures)

    for datum, trial_index in zip(data, trials_dict.keys()):
      if isnan(datum["packing_fraction"]):
        print(f"abandoning trial: {trial_index}")
        ax_client.abandon_trial(trial_index=trial_index)
        continue
      ax_client.complete_trial(trial_index=trial_index, raw_data=datum)

    if iter % checkpoint == 0:
      fpath = path.join(data_dir, f'sobol-{trial_index}-remove_composition_degeneracy={remove_composition_degeneracy},remove_scaling_degeneracy={remove_scaling_degeneracy}-checkpoint.json')
      ax_client.save_to_json_file(filepath=fpath)

best_parameters, metrics = ax_client.get_best_parameters()

In [ ]:
print(best_parameters)
print(metrics)

In [ ]:
fpath = path.join(data_dir, f'sobol-{num_trials}-remove_composition_degeneracy={remove_composition_degeneracy},remove_scaling_degeneracy={remove_scaling_degeneracy}.json')
ax_client.save_to_json_file(filepath=fpath)

## Code Graveyard

In [ ]:
# %cp /content/packing-generation/Docs/Examples/packing.xyzd /content/packing-generation/_Release/

In [ ]:
# import numpy as np
# from math import pi
# initial_density = sum(4/3 * pi * (X/2)**3) / box_length**3
# print("initial density: ", initial_density)
# desired_initial = 0.8
# vol_scale = desired_initial / initial_density
# d_scale = (6 * vol_scale / pi)**(1/3)
# print(d_scale)

In [ ]:
# !cp /content/packing-generation/_Release/PackingGeneration.exe /content/
# !./PackingGeneration.exe

In [ ]:
# """https://github.com/VasiliBaranov/packing-generation/issues/30#issue-1103925864"""
# import numpy as np
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# packing = np.fromfile('packing.xyzd').reshape(-1, 4)

# with open('packing.nfo', "r+") as nfo:
#     lines = nfo.readlines()
#     Theoretical_Porosity = float(lines[2].split()[2])
#     Final_Porosity = float(lines[3].split()[2])
#     # print(Theoretical_Porosity, Final_Porosity)

#     scaling_factor = ((1 - Final_Porosity) / (1 - Theoretical_Porosity)) ** (1 / 3)


#     real_diameters = packing[:, 3] * scaling_factor
#     actual_density = sum((4/3) * pi * (np.array(real_diameters) / 2)**3) / box_length**3
#     print(actual_density)
#     packing[:, 3] = real_diameters
#     packing.tofile('packing.xyzd')        # updating the packing: this line will modifies diameters in the packing.xyzd

#     # update packing.nfo and set TheoreticalPorosity to FinalPorosity to avoid scaling the packing once again the next time running this script.
#     lines[3] = lines[3].replace(str(Final_Porosity), str(Theoretical_Porosity))
#     nfo.seek(0)
#     nfo.writelines(lines)


# def ms(x, y, z, radius, resolution=10):
#     """Return the coordinates for plotting a sphere centered at (x,y,z)"""
#     u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
#     X = radius * np.cos(u)*np.sin(v) + x
#     Y = radius * np.sin(u)*np.sin(v) + y
#     Z = radius * np.cos(v) + z
#     return (X, Y, Z)

# df = pd.DataFrame(packing, columns=["x", "y", "z", "d"])
# df["r"] = df["d"] / 2
# print(df)

# # data = []
# # for index, row in df.iterrows():
# #     (x_pns_surface, y_pns_surface, z_pns_suraface) = ms(row.x, row.y, row.z, row.r)
# #     data.append(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_suraface, opacity=0.95))

# # fig = go.Figure(data=data)
# # fig.show()

In [ ]:
# from scipy.spatial.distance import pdist, squareform
# xyz = df[["x", "y", "z"]].values
# dm = pdist(xyz)
# min(dm / 2) - df.iloc[0]["r"]

# import plotly.express as px
# px.histogram(dm)

# from sklearn.neighbors import NearestNeighbors
# neigh = NearestNeighbors(n_neighbors=5, radius=box_length)
# neigh.fit(xyz)
# neigh_dist, neigh_ind = neigh.kneighbors(xyz)

# neigh_dist[:,1]

In [ ]:
# !git clone https://github.com/sparks-baird/bayes-opt-particle-packing.git --depth 1
# %mv bayes-opt-particle-packing/boppf/ ./

In [ ]:
# %cd 6efa1fff-0d6f-4577-be07-20e5555e63de
# !./PackingGeneration.exe -fba
# %cd /content/packing-generation/_Release

In [ ]:
# def robust_particle_packing_simulation(means, stds, comps, num_particles=100, contraction_rate=1e-3, seed=None, data_dir = ".", util_dir = ".", uid = None, retries=2, safety_factor_start = 1.5, safety_factor_end=3.0, num_safety_factor=5):
#   try:
#     actual_density = particle_packing_simulation(means, stds, comps, num_particles=num_particles, contraction_rate=contraction_rate, seed=seed, data_dir=data_dir, util_dir=util_dir, uid=uid, safety_factor=safety_factor_start)
#   except Exception as e:
#     for retry_safety_factor in np.linspace(safety_factor_start, safety_factor_end, num=num_safety_factor):
#       for retry in range(retries):
#         retry_seed = seed + 1000 * (1 + retry) if seed is not None else 1000 * (1 + retry)
#         try:
#           actual_density = particle_packing_simulation(means, stds, comps, num_particles=num_particles, contraction_rate=contraction_rate, safety_factor=retry_safety_factor, seed=retry_seed, data_dir=data_dir, util_dir=util_dir, uid=uid)
#         except:
#           if (retry_safety_factor == safety_factor_end) and (retry == retries - 1):
#             return np.nan

#   return actual_density

In [ ]:
    # trial_indices = list(trials_dict.keys())
    # remainders = [trial_index % checkpoint for trial_index in trial_indices]